In [14]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from crud_mod4 import CRUD  # Importing the CRUD class to interact with the database

###########################
# Data Manipulation / Model
###########################
# Database connection details
username = "aacuser"  # Database username
password = "SNHU1234"  # Database password
host = 'nv-desktop-services.apporto.com'  # Database host
port = 32748  # Database port
db = "AAC"  # Database name
collection = "animals"  # Collection name in the database

# Connect to the database via CRUD Module
shelter = CRUD(username, password, host, port, db, collection)  # Establish a connection to the database using CRUD class

# Read all records from the collection
df = pd.DataFrame.from_records(shelter.read({}))  # Retrieve all records and convert them into a pandas DataFrame

# Drop MongoDB '_id' field to avoid data table crash
if '_id' in df.columns:  # If the '_id' column exists in the data
    df.drop(columns=['_id'], inplace=True)  # Drop the '_id' column to prevent issues with rendering the data table

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)  # Initialize the Dash app

# Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'  # Path to the logo image file
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())  # Attempt to read and encode the logo image to base64
except FileNotFoundError:  # If the image is not found
    encoded_image = b""  # Set encoded_image to an empty byte string

# Define filter options (Example: Animal Outcome Type)
filter_options = [
    {'label': 'Water Rescue', 'value': 'Water Rescue'},
    {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
    {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
    {'label': 'Reset', 'value': 'Reset' }
]  # List of options for the rescue type filter

# Define the layout of the app
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 - Module 7 - Duane Wegner'))),  # Centered title in bold

    # Centered image using a container div
    html.Div(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '100px'}
        ),
        style={'textAlign': 'center'}
    ),

    html.Hr(),  # Horizontal rule (line) for visual separation

    # Interactive Filter
    html.Label("Select Rescue Type:"),  # Label for the filter
    dcc.RadioItems(
        id='filter-type',  # ID for the filter input
        options=filter_options,  # The filter options defined earlier
        value='Reset',  # Default selected value
        inline=True  # Display radio items inline (horizontally)
    ),

    html.Hr(),  # Another horizontal rule for visual separation

    # Interactive Data Table
    dash_table.DataTable(
        id='datatable-id',  # ID for the data table
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],  # Table columns
        data=df.to_dict('records'),  # Convert data to records format for the table
        filter_action="native",  # Allow native filtering in the table
        sort_action="native",  # Allow native sorting in the table
        sort_mode="multi",  # Enable multi-column sorting
        column_selectable="single",  # Allow single column selection
        row_selectable="single",  # Allow single row selection
        page_action="native",  # Use native pagination
        page_current=0,  # Set the initial page to 0
        page_size=10,  # Display 10 rows per page
        style_table={'overflowX': 'auto'}  # Style to allow horizontal scrolling if needed
    ),

    html.Br(),  # Line break for spacing
    html.Hr(),  # Another horizontal rule for visual separation

    # Layout for the graph and map side by side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),  # Placeholder for the graph (50% width)
        html.Div(id='map-id', className='col s12 m6')  # Placeholder for the map (50% width)
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on the selected filter
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])  # Trigger the callback when the filter selection changes
def update_dashboard(filter_type):
    if filter_type == "Water Rescue":
        data = shelter.read({"breed": {"$in": ["Labrador Retriever Mix"]}})  # Filter by breed for Water Rescue
    elif filter_type == "Mountain Rescue":
        data = shelter.read({"breed": {"$in": ["German Shepherd"]}})  # Filter by breed for Mountain Rescue
    elif filter_type == "Disaster Rescue":
        data = shelter.read({"breed": {"$in": ["Doberman Pinscher", "Golden Retriever"]}})  # Filter by breed for Disaster Rescue
    else:  # If Reset is selected, show all data
        data = shelter.read({})
    
    df = pd.DataFrame.from_records(data)  # Convert filtered data into a pandas DataFrame
    if '_id' in df.columns:  # Drop the '_id' column if present
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')  # Return the data in records format to update the table

# Callback to update the chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]  # Trigger the callback when data in the table changes
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)  # Convert the data into a pandas DataFrame
    if dff.empty or 'breed' not in dff.columns:  # If data is empty or 'breed' column is missing
        return html.P("No data to display chart.")  # Return a message if there's no data to display
    fig = px.pie(dff, names='breed', title='Breed Distribution')  # Create a pie chart showing breed distribution
    return [dcc.Graph(figure=fig)]  # Display the pie chart in the dashboard

# Callback to highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]  # Trigger when columns are selected
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},  # If the column is selected
        'background_color': '#D2F3FF'  # Highlight the selected column with a light blue background
    } for i in selected_columns or []]  # Apply the style to all selected columns

# Callback to display the map based on the selected data row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]  # Trigger when data or selected rows change
)
def update_map(viewData, index):
    if viewData is None or index is None:  # If there's no data or no row selected
        return []  # Return empty if no row is selected
    dff = pd.DataFrame.from_dict(viewData)  # Convert data into a pandas DataFrame
    if dff.empty or not index:  # If data is empty or no rows are selected
        return []  # Return empty
    row = index[0]  # Get the first selected row

    try:
        lat = dff.iloc[row]['location_lat']  # Get latitude of selected row
        lon = dff.iloc[row]['location_long']  # Get longitude of selected row
    except KeyError:  # If latitude or longitude is missing
        return [html.P("Location data not available.")]  # Return a message if location data is missing
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[  # Display map centered at location
            dl.TileLayer(id="base-layer-id"),  # Add base map layer
            dl.Marker(position=[lat, lon], children=[  # Add a marker at the location
                dl.Tooltip(dff.iloc[row]['breed']),  # Show breed name on hover
                dl.Popup([  # Add a popup with animal details
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the App
if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')


Connected to MongoDB
